In [12]:
import pandas as pd
import numpy as np
import time

In [ ]:
# raw_dt= pd.read_csv('data/202107-citibike-tripdata_pre_analyzed.csv')
# station_cluster = pd.read_csv('intersection_station_cluster.csv') 
# cluster_dict = dict(zip(station_cluster['station_id'], station_cluster['cluster']))

In [ ]:
# month_dt= raw_dt[(raw_dt['start_station_id'].isin(station_cluster['station_id']) )& (raw_dt['end_station_id'].isin(station_cluster['station_id']))]
# month_dt['start_station_id']= month_dt['start_station_id'].map(cluster_dict)
# month_dt['end_station_id']= month_dt['end_station_id'].map(cluster_dict)
# date_ls= sorted(list(month_dt['started_date'].unique())) 

In [13]:
paths = ['data/preprocessed/202102-citibike-tripdata_pre_analyzed.csv',
        'data/preprocessed/202103-citibike-tripdata_pre_analyzed.csv',
        'data/preprocessed/202104-citibike-tripdata_pre_analyzed.csv',
         'data/preprocessed/202105-citibike-tripdata_pre_analyzed.csv',
        'data/preprocessed/202106-citibike-tripdata_pre_analyzed.csv',
         'data/preprocessed/202107-citibike-tripdata_pre_analyzed.csv',
        'data/preprocessed/202108-citibike-tripdata_pre_analyzed.csv',
        'data/preprocessed/202109-citibike-tripdata_pre_analyzed.csv',
         'data/preprocessed/202110-citibike-tripdata_pre_analyzed.csv',
        'data/preprocessed/202111-citibike-tripdata_pre_analyzed.csv',
        'data/preprocessed/202112-citibike-tripdata_pre_analyzed.csv']

In [14]:
def create_hour_csv(path):
    
    raw_dt= pd.read_csv(path)
    station_cluster = pd.read_csv('intersection_station_cluster.csv') 
    cluster_dict = dict(zip(station_cluster['station_id'], 
                            station_cluster['cluster']))
    
    month_dt= raw_dt[(raw_dt['start_station_id'].isin(station_cluster['station_id']) )& (raw_dt['end_station_id'].isin(station_cluster['station_id']))]
    month_dt['start_station_id']= month_dt['start_station_id'].map(cluster_dict)
    month_dt['end_station_id']= month_dt['end_station_id'].map(cluster_dict)
    date_ls= sorted(list(month_dt['started_date'].unique())) 
    
    ##############################################################################
    
    month_matrix=[]
    hour_ls=[]
    full_date_ls=[]
    
    
    for date in date_ls:
        date_dt= month_dt[month_dt['started_date']==date]
        for hour in np.arange(start=0, stop= 24, step=2):
            hour_dt= date_dt[(date_dt['started_hour']<=hour+1) 
                             & (date_dt['started_hour']>= hour) 
                             &(date_dt['ended_hour']<=(hour+2))]
            count_hour_dt= hour_dt.groupby(by= ['start_station_id', 'end_station_id'], as_index=False)['ride_id'].count()
            for i in range(6):
                if (i not in set(count_hour_dt['start_station_id'])) or (i not in set(count_hour_dt['end_station_id'])):
                    count_hour_dt = count_hour_dt.append({'start_station_id':i, 'end_station_id':i, 'ride_id':-1},ignore_index=True)
            hour_pivot_dt= count_hour_dt.pivot(index='start_station_id', columns='end_station_id')['ride_id'].fillna(0)

            hour_pivot_dt[hour_pivot_dt<0] = 0
            hour_ls.append(hour)
            full_date_ls.append(date)
            month_matrix.append(hour_pivot_dt)
            hour_pivot_dt.to_csv('data/preprocessed/{d}-{h}.csv'.format(d = date, h=hour))


In [15]:
%load_ext autoreload
%autoreload 2
from multiprocessing import Pool
# from helper1 import transformation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Original Version

In [16]:
start = time.time()

for p in paths:
    create_hour_csv(p)

end = time.time()
print(end-start)

C:\ANACONDA\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-14-ce9d7ddeda55>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month_dt['start_station_id']= month_dt['start_station_id'].map(cluster_dict)
<ipython-input-14-ce9d7ddeda55>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month_dt['end_station_id']= month_dt['end_stat

C:\ANACONDA\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-14-ce9d7ddeda55>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month_dt['start_station_id']= month_dt['start_station_id'].map(cluster_dict)
<ipython-input-14-ce9d7ddeda55>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month_dt['end_station_id']= month_dt['end_stat

95.71109747886658


In [17]:
start = time.time()
with Pool(8) as pool: 
    re= pool.map(transformation, paths)
end = time.time()  

print(end-start)

24.61065697669983


In [18]:
# !zip -r /content/drive/MyDrive/3001_Proj/preprocessed/hour_csv.zip /content/drive/MyDrive/3001_Proj/preprocessed/hour_csv

In [19]:
# files.download('/content/drive/MyDrive/3001_Proj/preprocessed/hour_csv.zip')

In [ ]:
# exit()

In [ ]:
# for i in range(7):
#     if i not in set(count_hour_dt['start_station_id'].unique()):
#         count_hour_dt = count_hour_dt.append({'start_station_id':i, 'end_station_id':i, 'ride_id':-2},ignore_index=True)
# count_hour_dt
# hour_pivot_dt= count_hour_dt.pivot(index='start_station_id', columns='end_station_id')['ride_id'].fillna(0)
# hour_pivot_dt